# 3 register device to fiware on minikube

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/minikube/env

## register gamepad service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${GAMEPAD_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${GAMEPAD_TYPE}"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "_id": "5b557a594fe5ae0001ffb5a5",
  "subservice": "/gamepad",
  "service": "fiwaredemo",
  "apikey": "gamepad",
  "resource": "/iot/d",
  "attributes": [],
  "lazy": [],
  "commands": [],
  "entity_type": "gamepad",
  "internal_attributes": [],
  "static_attributes": []
}
```

## register gamepad device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${GAMEPAD_ID}",
      "entity_name": "${GAMEPAD_ID}",
      "entity_type": "${GAMEPAD_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
        {
          "name": "button",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/${GAMEPAD_ID}/ | jq .

example)
```json
{
  "device_id": "gamepad",
  "service": "fiwaredemo",
  "service_path": "/gamepad",
  "entity_name": "gamepad",
  "entity_type": "gamepad",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "button",
      "name": "button",
      "type": "string"
    }
  ],
  "lazy": [],
  "commands": [],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${GAMEPAD_ID}/ | jq .

example)
```json
{
  "id": "gamepad",
  "type": "gamepad",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "button": {
    "type": "string",
    "value": " ",
    "metadata": {}
  }
}
```

### register `cygnus` as a subscriber of gamepad device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${GAMEPAD_ID}.*",
      "type": "${GAMEPAD_TYPE}"
    }]
  },
  "notification": {
    "http": {
      "url": "http://cygnus:5050/notify"
    },
    "attrs": ["button"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5b4ef3b44d2c226e03a34f7b
fiware-correlator: b13811e4-8a60-11e8-8b4a-ae8577baef60
date: Wed, 18 Jul 2018 08:00:52 GMT
x-envoy-upstream-service-time: 3
server: envoy
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5b4ef5104d2c226e03a34f7c",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "gamepad.*",
          "type": "gamepad"
        }
      ],
      "condition": {
        "attrs": []
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2018-07-18T08:06:40.00Z",
      "attrs": [
        "button"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus:5050/notify"
      },
      "lastSuccess": "2018-07-18T08:06:40.00Z"
    }
  }
]
```

## register robot service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${ROBOT_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${ROBOT_TYPE}"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "_id": "5b557ac573d0e800012f2b86",
  "subservice": "/robot",
  "service": "fiwaredemo",
  "apikey": "robot",
  "resource": "/iot/d",
  "attributes": [],
  "lazy": [],
  "commands": [],
  "entity_type": "robot",
  "internal_attributes": [],
  "static_attributes": []
}
```

## register robot device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${ROBOT_ID}",
      "entity_name": "${ROBOT_ID}",
      "entity_type": "${ROBOT_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
        {
          "name": "x",
          "type": "float32"
        },
        {
          "name": "y",
          "type": "float32"
        },
        {
          "name": "z",
          "type": "float32"
        },
        {
          "name": "theta",
          "type": "float32"
        }
      ],
      "commands": [
        {
          "name": "move",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/${ROBOT_ID}/ | jq .

example)
```json
{
  "device_id": "turtlebot3",
  "service": "fiwaredemo",
  "service_path": "/robot",
  "entity_name": "turtlebot3",
  "entity_type": "robot",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "x",
      "name": "x",
      "type": "float32"
    },
    {
      "object_id": "y",
      "name": "y",
      "type": "float32"
    },
    {
      "object_id": "z",
      "name": "z",
      "type": "float32"
    },
    {
      "object_id": "theta",
      "name": "theta",
      "type": "float32"
    }
  ],
  "lazy": [],
  "commands": [
    {
      "object_id": "move",
      "name": "move",
      "type": "string"
    }
  ],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${ROBOT_ID}/ | jq .

example)
```json
{
  "id": "turtlebot3",
  "type": "robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "x": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "y": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "z": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### register `cygnus` as a subscriber of robot device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${ROBOT_ID}.*",
      "type": "${ROBOT_TYPE}"
    }]
  },
  "notification": {
    "http": {
      "url": "http://cygnus:5050/notify"
    },
    "attrs": ["x", "y", "z", "theta", "move_status", "move_info"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5b4ef3b44d2c226e03a34f7b
fiware-correlator: b13811e4-8a60-11e8-8b4a-ae8577baef60
date: Wed, 18 Jul 2018 08:00:52 GMT
x-envoy-upstream-service-time: 3
server: envoy
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5b4ef49847ebc6b99fae67dc",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "turtlebot3.*",
          "type": "robot"
        }
      ],
      "condition": {
        "attrs": []
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2018-07-18T08:04:40.00Z",
      "attrs": [
        "x",
        "y",
        "z",
        "theta",
        "move_status",
        "move_info"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus:5050/notify"
      }
    }
  }
]
```

## test subscribing the `button` attribute of `gamepad`

### prepare a command to emulate gamepad action

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+0900')
echo "mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${GAMEPAD_TYPE}/${GAMEPAD_ID}/attrs -m \"${d}|button|circle\""

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### send message to gamepad attribute topic

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate gamepad action`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.
example)
```
Client mosqsub|37715-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/gamepad/gamepad/atrs', ... (49 bytes))
2018-07-18T17:18:12.1531901892+0900|button|circle
```

### confirm gamepad entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${GAMEPAD_ID}/ | jq .

example)
```json
{
  "id": "gamepad",
  "type": "gamepad",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T17:24:09.1531902249+0900",
    "metadata": {}
  },
  "button": {
    "type": "string",
    "value": "circle",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:24:09.1531902249+0900"
      }
    }
  }
}
```

### confirm cygnus

In [ ]:
kubectl exec mongodb-0 -c mongodb -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${GAMEPAD_SERVICEPATH}_${GAMEPAD_ID}_${GAMEPAD_TYPE}\").find()"

example)
```
MongoDB shell version v3.6.6
connecting to: mongodb://127.0.0.1:27017/sth_fiwaredemo
MongoDB server version: 3.6.6
{ "_id" : ObjectId("5b4ef9353e9f0f000a6271e1"), "recvTime" : ISODate("2018-07-18T08:24:20.954Z"), "attrName" : "button", "attrType" : "string", "attrValue" : "circle" }
```

## test subscribing the `x, y, z, theta` attributes of `robot`

### prepare a command to emulate robot action

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+0900')
echo "mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${ROBOT_TYPE}/${ROBOT_ID}/attrs -m \"${d}|x|0.1|y|0.2|z|0.3|theta|0.4\""

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### send message to robot attribute topic

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate gamepad action`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.
example)
```
Client mosqsub|37876-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/robot/turtlebot3/attrs', ... (63 bytes))
2018-07-18T17:41:15.1531903275+0900|x|0.1|y|0.2|z|0.3|theta|0.4
```

### confirm robot entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${ROBOT_ID}/ | jq .

example)
```json
{
  "id": "turtlebot3",
  "type": "robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T17:41:15.1531903275+0900",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": "0.4",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "x": {
    "type": "float32",
    "value": "0.1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.2",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "z": {
    "type": "float32",
    "value": "0.3",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus

In [ ]:
OP_IN='$in'
kubectl exec mongodb-0 -c mongodb -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${ROBOT_SERVICEPATH}_${ROBOT_ID}_${ROBOT_TYPE}\").find({attrName: {${OP_IN}:[\"x\", \"y\", \"z\", \"theta\"]}})"

example)
```
MongoDB shell version v3.6.6
connecting to: mongodb://127.0.0.1:27017/sth_fiwaredemo
MongoDB server version: 3.6.6
{ "_id" : ObjectId("5b4efd473e9f0f000a6271e3"), "recvTime" : ISODate("2018-07-18T08:41:39.782Z"), "attrName" : "theta", "attrType" : "float32", "attrValue" : "0.4" }
{ "_id" : ObjectId("5b4efd473e9f0f000a6271e4"), "recvTime" : ISODate("2018-07-18T08:41:39.782Z"), "attrName" : "x", "attrType" : "float32", "attrValue" : "0.1" }
{ "_id" : ObjectId("5b4efd473e9f0f000a6271e5"), "recvTime" : ISODate("2018-07-18T08:41:39.782Z"), "attrName" : "y", "attrType" : "float32", "attrValue" : "0.2" }
{ "_id" : ObjectId("5b4efd473e9f0f000a6271e6"), "recvTime" : ISODate("2018-07-18T08:41:39.782Z"), "attrName" : "z", "attrType" : "float32", "attrValue" : "0.3" }
```

## test publishing the `move` command of `robot`

### prepare a command to emulate sending command

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
echo -e "curl -i -H \"Authorization: bearer ${TOKEN}\" -H \"Fiware-Service: ${FIWARE_SERVICE}\" -H \"Fiware-Servicepath: ${ROBOT_SERVICEPATH}\" -H \"Content-Type: application/json\" http://${HOST_IPADDR}:8080/orion/v1/updateContext -d @-<<__EOS__
{
  \"contextElements\": [
    {
      \"id\": \"${ROBOT_ID}\",
      \"isPattern\": \"false\",
      \"type\": \"${ROBOT_TYPE}\",
      \"attributes\": [
        {
          \"name\": \"move\",
          \"value\": \"square\"
        }
      ]
    }
  ],
  \"updateAction\": \"UPDATE\"
}
__EOS__"

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### send message to robot endpoint

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate sending command`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.
example)
```
Client mosqsub|38414-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/robot/turtlebot3/cmd', ... (22 bytes))
turtlebot3@move|square
```

### confirm robot entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${ROBOT_ID}/ | jq .

`move_status` is `PENDING`

example)
```json
{
  "id": "turtlebot3",
  "type": "robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T09:07:28.00Z",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T09:07:28.402Z"
      }
    }
  },
  "theta": {
    "type": "float32",
    "value": "0.4",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "x": {
    "type": "float32",
    "value": "0.1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.2",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "z": {
    "type": "float32",
    "value": "0.3",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### prepare a command to emulate receiving command result

In [ ]:
echo "mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${ROBOT_TYPE}/${ROBOT_ID}/cmdexe -m \"${ROBOT_ID}@move|executed square command\""

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### send message to robot cmdexe topic

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate receiving command result`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.
example)
```
Client mosqsub|38486-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/robot/turtlebot3/cmdexe', ... (39 bytes))
turtlebot3@move|executed square command
```

### confirm robot entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${ROBOT_ID}/ | jq .

`move_status` turns to `OK` and `move_info` has the result message

example)
```json
{
  "id": "turtlebot3",
  "type": "robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T09:15:26.00Z",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "executed square command",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T09:15:26.616Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T09:15:26.616Z"
      }
    }
  },
  "theta": {
    "type": "float32",
    "value": "0.4",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "x": {
    "type": "float32",
    "value": "0.1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.2",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "z": {
    "type": "float32",
    "value": "0.3",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus

In [ ]:
OP_IN='$in'
OP_AND='$and'
OP_NE='$ne'
kubectl exec mongodb-0 -c mongodb -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${ROBOT_SERVICEPATH}_${ROBOT_ID}_${ROBOT_TYPE}\").find({${OP_AND}: [{attrName: {${OP_IN}:[\"move_status\", \"move_info\"]}}, {attrValue:{${OP_NE}:\"UNKNOWN\"}}]})"

example)
```
MongoDB shell version v3.6.6
connecting to: mongodb://127.0.0.1:27017/sth_fiwaredemo
MongoDB server version: 3.6.6
{ "_id" : ObjectId("5b4f0350871cc3000a65cafe"), "recvTime" : ISODate("2018-07-18T09:07:28.402Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5b4f05303e9f0f000a6271e7"), "recvTime" : ISODate("2018-07-18T09:15:26.616Z"), "attrName" : "move_info", "attrType" : "commandResult", "attrValue" : "executed square command" }
{ "_id" : ObjectId("5b4f05303e9f0f000a6271e8"), "recvTime" : ISODate("2018-07-18T09:15:26.616Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "OK" }
```